In [1]:
# Montamos nuestro google drive en el colab

from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
from pathlib import Path
import pandas as pd

In [4]:
PATH = Path("/content/gdrive/My Drive/ml-practico-data")

In [5]:
# Clonamos el repo para usar el codigo de la lib
!git clone https://github.com/elsonidoq/ml-practico-2022.git
!cd ml-practico-2022; git pull

Cloning into 'ml-practico-2022'...
remote: Enumerating objects: 395, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 395 (delta 54), reused 49 (delta 23), pack-reused 313
Receiving objects: 100% (395/395), 4.85 MiB | 10.45 MiB/s, done.
Resolving deltas: 100% (245/245), done.
Already up to date.


In [6]:
import sys
sys.path.append('ml-practico-2022')

In [7]:
from lib import data, transformers
from lib.model import get_features_pipe, get_model_pipe

In [8]:
from tqdm.notebook import tqdm
import csv

In [24]:
import gzip
from gensim.models.callbacks import CallbackAny2Vec
from itertools import groupby
from io import TextIOWrapper

class IterSentences:
    def __init__(self, data_path, some=False):
        self.data_path = data_path
        self.some = some
        
    def __iter__(self):
        reader = csv.DictReader(TextIOWrapper(gzip.open(self.data_path / 'title.principals.tsv.gz')), delimiter='\t')
        for i, (tconst, rows) in enumerate(groupby(tqdm(reader), lambda x: x['tconst'])):
            if self.some and i == 10000: break
            yield [e['nconst'] for e in sorted(rows, key=lambda x: int(x['ordering']))]
            

class EpochSaver(CallbackAny2Vec):
    def __init__(self, data_path, fname_prefix=''):
        self.data_path = data_path
        self.fname_prefix = fname_prefix
        self.epoch = 1
    
    def on_epoch_end(self, model):
        print(f'Finished epoch {self.epoch}. Saving...')
        prefix = f'{self.fname_prefix}_' if self.fname_prefix else ''
        
        output_path = self.data_path / 'w2v' / f'{prefix}epoch_{self.epoch}'
        output_path.parent.mkdir(parents=True, exist_ok=True)
        
        model.save(str(output_path))
        self.epoch += 1

In [25]:
from gensim.models import Word2Vec

In [26]:
model = Word2Vec(window=10, iter=10,  callbacks=[EpochSaver(PATH)])

In [27]:
model.build_vocab(IterSentences(PATH, some=False))

0it [00:00, ?it/s]

In [28]:
model.train(IterSentences(PATH), total_words=model.corpus_total_words, epochs=10)

0it [00:00, ?it/s]

Finished epoch 1. Saving...


0it [00:00, ?it/s]

Finished epoch 2. Saving...


0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

Finished epoch 9. Saving...


0it [00:00, ?it/s]

Finished epoch 10. Saving...


(453556130, 508249070)